# RNN Time-Series Forecasting (Weather Dataset example)

## IADS Summer School, 3rd August 2021

### Dr Michael Fairbank, University of Essex, UK

- Email: m.fairbank@essex.ac.uk
- This is Jupyter Notebook 3.1 of the course

- We will use a RNN to forecast the future weather temperature, based on previous weather observations.

### Acknowlegements: 
- This is a modified notebook of the example from https://www.tensorflow.org/tutorials/structured_data/time_series by Francois Chollet. 

## Load weather dataset

- We use Pandas to load the csv file very easily in Python.  Pandas also lets us easily manipulate and view the data.

- This dataset used is a <a href="https://www.bgc-jena.mpg.de/wetter/" class="external">weather time series dataset</a> recorded by the <a href="https://www.bgc-jena.mpg.de" class="external">Max Planck Institute for Biogeochemistry</a>. This dataset contains 14 different features such as air temperature, atmospheric pressure, and humidity. These were collected every 10 minutes, beginning in 2003. For efficiency, you will use only the data collected between 2009 and 2016. This section of the dataset was prepared by François Chollet for his book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python).



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

df = pd.read_csv("datasets/weather_data.csv")
# If the above line fails then try the following line instead, to download it directly from 
# my google drive account
# df = pd.read_csv("https://drive.google.com/uc?export=download&id=1Xfst8BDlkOlLzL3ArfBm90q6zUm9isgk")


- Let's take a glance at the data. Here are the first few rows.  To save us time, I've already preprocessed it a bit (following the method of the above tutorial.)


In [ ]:
print(df.head())


- Note the Day Sin and Day Cos columns have been pre-calculated here as sin(clock_hour\/24\*2\*PI), cos(clock_hour/24\*2\*PI), since this gives massive useful hints as to how the sun-shine cycles overhead each day.

- Similarly, the Year Sin and Year Cos columns have been pre-calculated as sin(month_in_year\/12\*2\*PI) etc, which gives strong hints about the yearly seasonal cycles.

**Question:**  Why do we use sin(clock_hour\/24\*PI) as opposed to just directly using clock_hour?  Discuss this with the person next to you and come up with an agreed answer.

## View portion of source data graphically

In [ ]:
plot_cols = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols][:480] # just plotting first 480 data points here
plot_features.index = df["Date Time"][:480]
_ = plot_features.plot(subplots=True)

## Split the Data

We'll use a `(70%, 30%)` split for the training and validation. Note the data is **not** being randomly shuffled before splitting. This is for two reasons.

1. It ensures that chopping the data into windows of consecutive samples is still possible.
2. It ensures that the validation/test results are more realistic, being evaluated on data collected after the model was trained.

In [ ]:
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):n]


### Normalize the data

It is important to scale features before training a neural network. Normalization is a common way of doing this scaling. Subtract the mean and divide by the standard deviation of each feature.

The mean and standard deviation should only be computed using the training data so that the models have no access to the values in the validation (and test) sets.


In [ ]:
train_df.pop("Date Time") # remove this since it is a string column and will mess up the normalisation
val_df.pop("Date Time") # remove this since it is a string column and will mess up the normalisation

#Normalise input data:   Note we have to use the mean of the training set only, even for normalisation of the other 2 datasets
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
print("Normalised input data", train_df.head().transpose())

## Choose input columns and target output columns for forecasting objective:

Here we decide which columns to show the neural network, and which columns we'd like it to be trained to predict.

In [ ]:
input_columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
target_columns=[1] # This is temperature column
print("Input columns", list(train_df.columns[input_columns]))
print("Target columns", list(train_df.columns[target_columns]))

## Prepare input and output time sequences

We want the RNN to receive a stream of data, corresponding to the above input columns, spread over time, and to make a forecast of the target column quantity for look_ahead timesteps ahead.

We will prepare this data as 3d tensors suitable for input to Keras RNNs, so of shape [batch_size, time_sequence_length, data_width]

Note we say a time_sequence_length=10, and allow a warm_up_period=4, which gives the RNN 4 timesteps to observe the context on what the time series is doing before it is being challenged to make any predictions.  It will then have to make 10-4=6 consecutive predictions, where each prediction is look_head=1 hour.

In [ ]:
time_sequence_length=10
warm_up_period=4
look_ahead=1

def generate_input_and_target_tensor(df, input_cols, target_col, time_sequence_length, warm_up_period, look_ahead):
    num_rows=len(df)
    data=np.array(df,np.float32)
    inputs=[]
    targets=[]
    for first_row in range(num_rows-time_sequence_length-look_ahead):
        inputs.append(data[first_row:first_row+time_sequence_length,input_cols])
        targets.append(data[first_row+look_ahead+warm_up_period:first_row+look_ahead+time_sequence_length,target_col])
    return np.stack(inputs), np.stack(targets)
[inputs_tensor_train,targets_tensor_train]=generate_input_and_target_tensor(train_df, input_columns, target_columns, time_sequence_length, warm_up_period, look_ahead)
[inputs_tensor_val,targets_tensor_val]=generate_input_and_target_tensor(val_df, input_columns, target_columns, time_sequence_length, warm_up_period, look_ahead)

print("inputs",inputs_tensor_train.shape,"[batch, time_sequence_length, num_inputs_per_timestep]")
print("targets",targets_tensor_train.shape,"[batch, time_sequence_length-warm_up_period, num_outputs_per_timestep]")

## Build RNN Model (LSTM - With return_sequences=True)

- Note that in this LSTM, the parameter return_sequences=True.  This changes the RNN into a "many-to-many" RNN.
    - In a many-to-many RNN, the extra red outputs are included.  
    - In a many-to-one RNN the red outputs are omitted:
<img src="./images/rnn_unrolled_many_to_many.svg"  width="400">

- This means the LSTM layer receives as input a rank-3 tensor, and gives a rank-3 tensor as output too:
<img src="./images/simplernn_layer_network_return_sequences.svg"  width="600">

- This is like a previous diagram we saw, but now, because return_sequences=True, the output of the LSTM layer is rank-3.
- Note that a Dense layer can act on a Rank-3 input itself to give a rank-3 output.
- Hence this RNN takes as input a rank-3 tensor and gives a rank-3 tensor output.

Also note,
- The code below uses a 3rd way to define a keras model - the "Keras Functional API" method.  See here for more details https://www.tensorflow.org/guide/keras/functional

In [ ]:
#build model
inputs = keras.Input(shape=(time_sequence_length,len(input_columns)), name='input')
#hidden_vectors3d = layers.SimpleRNN(20, return_sequences=True,activation='tanh')(inputs)
hidden_vectors3d = layers.LSTM(20, return_sequences=True,activation='tanh')(inputs)
if warm_up_period>0:
    # Chop off the first warm_up_period time steps from the series.
    hidden_vectors3d=hidden_vectors3d[:,warm_up_period:,:]
output_vectors3d=layers.Dense(len(target_columns))(hidden_vectors3d)
model = keras.Model(inputs=inputs, outputs=output_vectors3d)
model.summary()

## Compile Model

Specify the optimiser (Adam) and the loss function to use.  This is a regression task (we are forecasting a real number) so we use MeanSquaredError.

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(0.01), 
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[tf.metrics.MeanAbsoluteError()],
)

## Train Model

In [ ]:
history = model.fit(
    inputs_tensor_train,
    targets_tensor_train,
    batch_size=100,
    epochs=10,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(inputs_tensor_val, targets_tensor_val),
)

## Visualise training progress
- We can use these graphs to decide how much "overfitting" might have happened.

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"],label="Val loss")
plt.legend()
plt.grid()

# View some individual forecasts

Let's print some specific forecast time-series, and compare them to their targets.

- Note: We have to reverse the standard-deviation normalisation used in the data-preparation here to recover actual temperatures.

In [ ]:
train_std=np.array(train_std[target_columns])
train_mean=np.array(train_mean[target_columns])
for row in range(0,40,4):
    print(row,"targets",(targets_tensor_val[row,:,0]*train_std+train_mean))
    print(row,"prediction",(model(inputs_tensor_val[row:row+1]).numpy()[0,:,0]*train_std+train_mean))
    

- Hopefully some of the above numbers match reasonably closely to convince us that the forecasting is working reasonably well.  For a better view, we plot a graph next:

# Plot forecast graph

Let's try to show graphically the first 40 points forecast in the validation set, and compare to their targets.

In [ ]:
targets_=[]
predictions_=[]
for row in range(40):
    targets_.append(targets_tensor_val[row,-1,0]*train_std+train_mean) # here we are truncating the time series so tha only the final timestep is selected
    predictions_.append(model(inputs_tensor_val[row:row+1]).numpy()[0,-1,0]*train_std+train_mean)
plt.ylabel(train_df.columns[target_columns][0])
plt.plot(range(len(targets_)), targets_,label='Labels', marker='.', zorder=-10)
plt.scatter(range(len(predictions_)), predictions_,edgecolors='k', label='Predictions', c='#2ca02c', s=64)
plt.legend()
plt.xlabel('Time step [h] (forecast length='+str(look_ahead)+'h)')
plt.show()

Another way to view the accuracy of the model, over a larger range of data points, is to plot a scatter graph of actual temperature versus forecast temperature.  These should line up on $y=x$ if it's working well.

In [ ]:
targets=(targets_tensor_val[:,-1,0]*train_std+train_mean) 
predictions=(model(inputs_tensor_val[:]).numpy()[:,-1,0]*train_std+train_mean)
plt.scatter(targets, predictions, label="Predictions")
plt.xlabel("Target")
plt.ylabel("Prediction")
plt.legend()

## Extra challenges

1. These graphs looks good.  What happens if we change the forecast length to 4h?  Will it still work?  Try it. 

2. Can you add another scatter plot like the one above, but which shows the forecast temperature *change* versus the actual temperature *change* (where "change" here means the (temperature at time (t+look_ahead))-(temperature at time t)).

3. How does performance change if we change the LSTM layer to a SimpleRNN layer?

4. Also you can change the number of time steps the RNN gets to look at before it has to make a forecast.

5. Which inputs were most helpful?  Presumably clock time in the day, and month of the year, is very useful for weather forecasting.  Try removing some inputs to see which ones were most helpful.

6. Research tf.keras.layers.Dropout and tf.keras.callbacks.EarlyStopping with keras.  
    - Try adding a "dropout" layer and/or "early stopping" to try to improve performance.

## Further work

- In the next exercise we will try this method on a financial forecasting problem

- To build more varied weather forecasing models you could change the inputs used.  See the original tutorial on which this demo was based to see how to modify the RNN to feed forecasts back into itself so forecasts can be made on forecasts.